In [1]:
import sympy as sym
import numpy as np
import time

from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Operator
from qiskit.extensions import RXGate, RYGate, RZGate  

In [2]:
def state_out_real(dx,dy,dz):     
    I_f = np.array([[1, 0],
                  [0, 1]]) 
    I = np.array([[1, 0],
                  [0, 1]])
    X_f = np.array([[0, 1],
                  [1, 0]]) 
    X = np.array([[0, 1],
                  [1, 0]])   

    for q in range(1):
        I_f = np.kron(I_f, I)
        X_f = np.kron(X_f, X)
    J = Operator(1 / np.sqrt(2) * (I_f + 1j * X_f))    
    J_dg = J.adjoint()
    circ = QuantumCircuit(2,2)
    circ.append(J, range(2))

    for q in range(2):
        circ.append(RXGate(dx),[q])
        circ.append(RYGate(dy),[q])
        circ.append(RZGate(dz),[q])    

    circ.append(J_dg, range(2))

    backend = Aer.get_backend('statevector_simulator')
    job = backend.run(circ)
    result = job.result()
    outputstate = result.get_statevector(circ, decimals=5)

    return np.round(np.array(outputstate).reshape(4,1),5)

In [17]:
"""state_00 = np.array([1, 0, 0, 0]).reshape(4,1)

I = np.matrix([[1, 0],
              [0, 1]])
X = np.matrix([[0, 1],
              [1, 0]])    
I_f = np.kron(I, I)
X_f = np.kron(X, X)
J = 1 / np.sqrt(2) * (I_f + 1j * X_f)
J_dg = J.H

x = sym.Symbol('x')
y = sym.Symbol('y')
z = sym.Symbol('z')
Rx = sym.Matrix([[sym.cos(x/2), -sym.I * sym.sin(x/2)], [-sym.I * sym.sin(x/2), sym.cos(x/2)]])
Ry = sym.Matrix([[sym.cos(y/2), -sym.sin(y/2)], [sym.sin(y/2), sym.cos(y/2)]])
Rz = sym.Matrix([[sym.exp(-sym.I * z/2), 0], [0, sym.exp(sym.I * z/2)]])
G_Rx = np.kron(Rx, Rx)
G_Ry = np.kron(Ry, Ry)
G_Rz = np.kron(Rz, Rz)

state_f = (J_dg * G_Rz * G_Ry * G_Rx * J * state_00)
#print(state_f)"""

def state_out_new(x,y,z):

    sqrt_2 = np.sqrt(2)
    cos_x = np.cos(x/2)
    sin_x = np.sin(x/2)
    cos_y = np.cos(y/2)
    sin_y = np.sin(y/2)
    exp_z = np.exp(1j*z)
    exp_m = np.exp(-1j*z)

    s_0 = sqrt_2/2*(-sqrt_2/2*1j*exp_z*sin_y**2 + sqrt_2/2*exp_m*cos_y**2)*cos_x**2 - sqrt_2/2*(-sqrt_2/2*1j*exp_z*cos_y**2 + sqrt_2/2*exp_m*sin_y**2)*sin_x**2 - sqrt_2*1j*(-sqrt_2/2*1j*exp_z*sin_y*cos_y - sqrt_2/2*exp_m*sin_y*cos_y)*sin_x*cos_x + sqrt_2/2*1j*(-(-sqrt_2/2*1j*exp_z*sin_y**2 + sqrt_2/2*exp_m*cos_y**2)*sin_x**2 + (-sqrt_2/2*1j*exp_z*cos_y**2 + sqrt_2/2*exp_m*sin_y**2)*cos_x**2 - 2*1j*(-sqrt_2/2*1j*exp_z*sin_y*cos_y - sqrt_2/2*exp_m*sin_y*cos_y)*sin_x*cos_x)
    s_1 = -sqrt_2/2*1j*(sqrt_2/2*1j*sin_y**2 + sqrt_2/2*cos_y**2)*sin_x*cos_x - sqrt_2/2*(-sqrt_2/2*sin_y*cos_y + sqrt_2/2*1j*sin_y*cos_y)*sin_x**2 + sqrt_2/2*(sqrt_2/2*sin_y*cos_y - sqrt_2/2*1j*sin_y*cos_y)*cos_x**2 - sqrt_2/2*1j*(-sqrt_2/2*sin_y**2 - sqrt_2/2*1j*cos_y**2)*sin_x*cos_x + sqrt_2/2*1j*(-1j*(sqrt_2/2*1j*sin_y**2 + sqrt_2/2*cos_y**2)*sin_x*cos_x + (-sqrt_2/2*sin_y*cos_y + sqrt_2/2*1j*sin_y*cos_y)*cos_x**2 - (sqrt_2/2*sin_y*cos_y - sqrt_2/2*1j*sin_y*cos_y)*sin_x**2 - 1j*(-sqrt_2/2*sin_y**2 - sqrt_2/2*1j*cos_y**2)*sin_x*cos_x)
    s_2 = -sqrt_2/2*1j*(sqrt_2/2*1j*sin_y**2 + sqrt_2/2*cos_y**2)*sin_x*cos_x - sqrt_2/2*(-sqrt_2/2*sin_y*cos_y + sqrt_2/2*1j*sin_y*cos_y)*sin_x**2 + sqrt_2/2*(sqrt_2/2*sin_y*cos_y - sqrt_2/2*1j*sin_y*cos_y)*cos_x**2 - sqrt_2/2*1j*(-sqrt_2/2*sin_y**2 - sqrt_2/2*1j*cos_y**2)*sin_x*cos_x + sqrt_2/2*1j*(-1j*(sqrt_2/2*1j*sin_y**2 + sqrt_2/2*cos_y**2)*sin_x*cos_x + (-sqrt_2/2*sin_y*cos_y + sqrt_2/2*1j*sin_y*cos_y)*cos_x**2 - (sqrt_2/2*sin_y*cos_y - sqrt_2/2*1j*sin_y*cos_y)*sin_x**2 - 1j*(-sqrt_2/2*sin_y**2 - sqrt_2/2*1j*cos_y**2)*sin_x*cos_x)
    s_3 = sqrt_2/2*(sqrt_2/2*exp_z*sin_y**2 - sqrt_2/2*1j*exp_m*cos_y**2)*cos_x**2 - sqrt_2/2*(sqrt_2/2*exp_z*cos_y**2 - sqrt_2/2*1j*exp_m*sin_y**2)*sin_x**2 - sqrt_2*1j*(sqrt_2/2*exp_z*sin_y*cos_y + sqrt_2/2*1j*exp_m*sin_y*cos_y)*sin_x*cos_x + sqrt_2/2*1j*(-(sqrt_2/2*exp_z*sin_y**2 - sqrt_2/2*1j*exp_m*cos_y**2)*sin_x**2 + (sqrt_2/2*exp_z*cos_y**2 - sqrt_2/2*1j*exp_m*sin_y**2)*cos_x**2 - 2*1j*(sqrt_2/2*exp_z*sin_y*cos_y + sqrt_2/2*1j*exp_m*sin_y*cos_y)*sin_x*cos_x)
    p_new = np.round(np.array([s_0, s_1, s_2, s_3]).reshape(1,4),5)
    p_new = np.abs(p_new)**2
    
    return (p_new)/(np.sum(p_new))

In [19]:
state_new = state_out_new(0.7853981633974483, 3.141592653589793, 0)[0]
print(np.sum(state_new))

0.9999999999999999


In [4]:
def state_out_new_2(x,y,z):

    s_0 = np.sqrt(2)/2*(-np.sqrt(2)/2*1j*np.exp(1j*z)*np.sin(y/2)**2 + np.sqrt(2)/2*np.exp(-1j*z)*np.cos(y/2)**2)*np.cos(x/2)**2 - np.sqrt(2)/2*(-np.sqrt(2)/2*1j*np.exp(1j*z)*np.cos(y/2)**2 + np.sqrt(2)/2*np.exp(-1j*z)*np.sin(y/2)**2)*np.sin(x/2)**2 - np.sqrt(2)*1j*(-np.sqrt(2)/2*1j*np.exp(1j*z)*np.sin(y/2)*np.cos(y/2) - np.sqrt(2)/2*np.exp(-1j*z)*np.sin(y/2)*np.cos(y/2))*np.sin(x/2)*np.cos(x/2) + np.sqrt(2)/2*1j*(-(-np.sqrt(2)/2*1j*np.exp(1j*z)*np.sin(y/2)**2 + np.sqrt(2)/2*np.exp(-1j*z)*np.cos(y/2)**2)*np.sin(x/2)**2 + (-np.sqrt(2)/2*1j*np.exp(1j*z)*np.cos(y/2)**2 + np.sqrt(2)/2*np.exp(-1j*z)*np.sin(y/2)**2)*np.cos(x/2)**2 - 2*1j*(-np.sqrt(2)/2*1j*np.exp(1j*z)*np.sin(y/2)*np.cos(y/2) - np.sqrt(2)/2*np.exp(-1j*z)*np.sin(y/2)*np.cos(y/2))*np.sin(x/2)*np.cos(x/2))
    s_1 = -np.sqrt(2)/2*1j*(np.sqrt(2)/2*1j*np.sin(y/2)**2 + np.sqrt(2)/2*np.cos(y/2)**2)*np.sin(x/2)*np.cos(x/2) - np.sqrt(2)/2*(-np.sqrt(2)/2*np.sin(y/2)*np.cos(y/2) + np.sqrt(2)/2*1j*np.sin(y/2)*np.cos(y/2))*np.sin(x/2)**2 + np.sqrt(2)/2*(np.sqrt(2)/2*np.sin(y/2)*np.cos(y/2) - np.sqrt(2)/2*1j*np.sin(y/2)*np.cos(y/2))*np.cos(x/2)**2 - np.sqrt(2)/2*1j*(-np.sqrt(2)/2*np.sin(y/2)**2 - np.sqrt(2)/2*1j*np.cos(y/2)**2)*np.sin(x/2)*np.cos(x/2) + np.sqrt(2)/2*1j*(-1j*(np.sqrt(2)/2*1j*np.sin(y/2)**2 + np.sqrt(2)/2*np.cos(y/2)**2)*np.sin(x/2)*np.cos(x/2) + (-np.sqrt(2)/2*np.sin(y/2)*np.cos(y/2) + np.sqrt(2)/2*1j*np.sin(y/2)*np.cos(y/2))*np.cos(x/2)**2 - (np.sqrt(2)/2*np.sin(y/2)*np.cos(y/2) - np.sqrt(2)/2*1j*np.sin(y/2)*np.cos(y/2))*np.sin(x/2)**2 - 1j*(-np.sqrt(2)/2*np.sin(y/2)**2 - np.sqrt(2)/2*1j*np.cos(y/2)**2)*np.sin(x/2)*np.cos(x/2))
    s_2 = -np.sqrt(2)/2*1j*(np.sqrt(2)/2*1j*np.sin(y/2)**2 + np.sqrt(2)/2*np.cos(y/2)**2)*np.sin(x/2)*np.cos(x/2) - np.sqrt(2)/2*(-np.sqrt(2)/2*np.sin(y/2)*np.cos(y/2) + np.sqrt(2)/2*1j*np.sin(y/2)*np.cos(y/2))*np.sin(x/2)**2 + np.sqrt(2)/2*(np.sqrt(2)/2*np.sin(y/2)*np.cos(y/2) - np.sqrt(2)/2*1j*np.sin(y/2)*np.cos(y/2))*np.cos(x/2)**2 - np.sqrt(2)/2*1j*(-np.sqrt(2)/2*np.sin(y/2)**2 - np.sqrt(2)/2*1j*np.cos(y/2)**2)*np.sin(x/2)*np.cos(x/2) + np.sqrt(2)/2*1j*(-1j*(np.sqrt(2)/2*1j*np.sin(y/2)**2 + np.sqrt(2)/2*np.cos(y/2)**2)*np.sin(x/2)*np.cos(x/2) + (-np.sqrt(2)/2*np.sin(y/2)*np.cos(y/2) + np.sqrt(2)/2*1j*np.sin(y/2)*np.cos(y/2))*np.cos(x/2)**2 - (np.sqrt(2)/2*np.sin(y/2)*np.cos(y/2) - np.sqrt(2)/2*1j*np.sin(y/2)*np.cos(y/2))*np.sin(x/2)**2 - 1j*(-np.sqrt(2)/2*np.sin(y/2)**2 - np.sqrt(2)/2*1j*np.cos(y/2)**2)*np.sin(x/2)*np.cos(x/2))
    s_3 = np.sqrt(2)/2*(np.sqrt(2)/2*np.exp(1j*z)*np.sin(y/2)**2 - np.sqrt(2)/2*1j*np.exp(-1j*z)*np.cos(y/2)**2)*np.cos(x/2)**2 - np.sqrt(2)/2*(np.sqrt(2)/2*np.exp(1j*z)*np.cos(y/2)**2 - np.sqrt(2)/2*1j*np.exp(-1j*z)*np.sin(y/2)**2)*np.sin(x/2)**2 - np.sqrt(2)*1j*(np.sqrt(2)/2*np.exp(1j*z)*np.sin(y/2)*np.cos(y/2) + np.sqrt(2)/2*1j*np.exp(-1j*z)*np.sin(y/2)*np.cos(y/2))*np.sin(x/2)*np.cos(x/2) + np.sqrt(2)/2*1j*(-(np.sqrt(2)/2*np.exp(1j*z)*np.sin(y/2)**2 - np.sqrt(2)/2*1j*np.exp(-1j*z)*np.cos(y/2)**2)*np.sin(x/2)**2 + (np.sqrt(2)/2*np.exp(1j*z)*np.cos(y/2)**2 - np.sqrt(2)/2*1j*np.exp(-1j*z)*np.sin(y/2)**2)*np.cos(x/2)**2 - 2*1j*(np.sqrt(2)/2*np.exp(1j*z)*np.sin(y/2)*np.cos(y/2) + np.sqrt(2)/2*1j*np.exp(-1j*z)*np.sin(y/2)*np.cos(y/2))*np.sin(x/2)*np.cos(x/2))
    p_new = np.round(np.array([s_0, s_1, s_2, s_3]).reshape(4,1),5)
    
    return p_new

In [6]:
angulos = np.arange(0, 2 * np.pi, 2 * np.pi / 8)
counter = 0

for rx in angulos:
    for ry in angulos:
        for rz in angulos:
            state_real = state_out_real(rx,ry,rz)
            state_new = state_out_new_2(rx,ry,rz)
            if (state_real == state_new).all:
                counter += 1
                
print("Accuracy = {}".format(counter/np.power(len(angulos),3)))                

Accuracy = 1.0


In [7]:
angulos = np.arange(0, 2 * np.pi, 2 * np.pi / 8)

t_0_real = time.time()
for rx in angulos:
    for ry in angulos:
        for rz in angulos:
            state_real = state_out_real(rx,ry,rz)
t_f_real = time.time() - t_0_real
print("Tiempo de procesamiento en Qiskit = {}".format(t_f_real))       

t_0_new = time.time()
for rx in angulos:
    for ry in angulos:
        for rz in angulos:
            state_new = state_out_new(rx,ry,rz)
t_f_new = time.time() - t_0_new
print("Tiempo de procesamiento nuevo = {}".format(t_f_new))      

t_0_new2 = time.time()
for rx in angulos:
    for ry in angulos:
        for rz in angulos:
            state_new = state_out_new_2(rx,ry,rz)
t_f_new2 = time.time() - t_0_new2
print("Tiempo de procesamiento nuevo = {}".format(t_f_new2))        

Tiempo de procesamiento en Qiskit = 1.3950862884521484
Tiempo de procesamiento nuevo = 0.2347698211669922
Tiempo de procesamiento nuevo = 0.46165037155151367
